## Experiment-06
### Amirreza Fosoul and Bithiah Yuan

In [118]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import *
import string
import csv
import re
import time
from pyspark.sql.functions import *
from pyspark.sql.functions import split
import pyspark.sql.functions as f
import re
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType, DoubleType
from pyspark.sql.functions import desc
from pyspark.sql.window import Window as W
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.functions import concat, col, lit, size
from pyspark.sql.types import ArrayType
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import IntegerType
import random
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import math
import re
from pyspark.ml.linalg import SparseVector, VectorUDT, DenseVector
import numpy as np

spark = SparkSession.builder.appName('ex6').getOrCreate()
sc = spark.sparkContext


In [2]:
# Unexplode a column to an array of given datatype
def unExplode(df, groupByColName, collectColName, colType):
    types = {'string': StringType(), 'integer': IntegerType()}
    df_collected = df.groupby(groupByColName).agg(f.concat_ws(", ", f.collect_list(df[collectColName])).alias(collectColName))
    result = df_collected.withColumn(collectColName, split(col(collectColName), ",\s*").cast(ArrayType(types[colType])).alias(collectColName)).orderBy(groupByColName)
    return result

In [3]:
# Read user ratings into Dataframe
user_df = spark.read.option("delimiter", ";").csv('./example0.txt')
#user_df = spark.read.option("delimiter", ";").csv('./example.txt')
user_df = user_df.select(col("_c0").alias("userID"), col("_c1").alias("paperID"))

user_pre = user_df

user_df = user_df.select("userID", f.split("paperID", ",").alias("papers"), f.explode(f.split("paperID", ",")).alias("paperID"))
user_df = user_df.drop("papers")

# Count the number of distinct users
numUsers = user_df.select("userID").distinct().count()

# Get a dataframe of the distinct papers
d_paper = user_df.select("paperID").distinct()

userLib = user_pre.withColumn("paperID", split(col("paperID"), ",").cast(ArrayType(IntegerType()))).orderBy("userID")

get_len_udf = udf(lambda x: len(x), IntegerType())

userLib = userLib.withColumn("libSize", get_len_udf(col("paperID")))

userLib.show()

+--------------------+--------------------+-------+
|              userID|             paperID|libSize|
+--------------------+--------------------+-------+
|589b870a611c25fa9...|[12832332, 130547...|      5|
|90f1a3e6fcdbf9bc5...|[115945, 11733005...|      5|
+--------------------+--------------------+-------+



In [143]:
# Read in the stopwords as a list
with open('./stopwords_en.txt') as file:
    stopwordList = file.read().splitlines()
    
# Read in records of paper information
#w_df = spark.read.csv('./papers.csv')
w_df = spark.read.csv('./paper0.csv')
#w_df = w_df.select("_c0", "_c01" "_c06", "_c09", "_c13", "_c14")
w_df = w_df.select("_c0", "_c1", "_c6", "_c9", "_c13", "_c14")
#w_df = w_df.select(col("_c0").alias("paperID"), col("_c01").alias("type"), col("_c06").alias("pages"), col("_c09").alias("year"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.select(col("_c0").alias("paperID"), col("_c1").alias("type"), col("_c6").alias("pages"), col("_c9").alias("year"), col("_c13").alias("title"), col("_c14").alias("abstract"))
w_df = w_df.na.fill({'title': '', 'abstract': ''}) # to replace null values with empty string
# Get text from title and abstract
w_df = w_df.select(col("paperID"), "type", "pages", "year", concat(col("title"), lit(" "), col("abstract")).alias("words"))
w_df.show()

+--------+-------------+-----+----+--------------------+
| paperID|         type|pages|year|               words|
+--------+-------------+-----+----+--------------------+
|12832332|       inbook|   29|2013|deepening democra...|
| 1305474|         misc| null|2002|the {tesla} broad...|
| 1001231|  proceedings| null|2005|anonymous communi...|
|  352713|         misc|   12|2002|a survey on senso...|
|  956315|      article|   10|2006|wormhole attacks ...|
|  945604|inproceedings| null|2003|packet leashes: a...|
|10294999|      article| null|2012|a 2\% distance to...|
|  967275|inproceedings|   11|2005|on the survivabil...|
|  115945|      article|   11|2005|a twelve-step pro...|
|11733005|      article|    7|2012|evidence for wide...|
| 9045137|      article|   24|2007|nuclear {dna} con...|
| 3728173|      article|    5|2007|evolution of indi...|
| 8310458|      article|   15|2008|evolution of comp...|
|   80546|      article|   17|2004|the arbitrariness...|
| 5842862|      article|    2|2

### Exercise 6. 1 (Problem Modeling)

In [53]:
# Create a user hash map dataframe by adding a column with descending row numbers 
# to the pre-explode user dataframe
# user_hash_map = user_pre.withColumn("user_idx", F.monotonically_increasing_id().cast(IntegerType()))

# Select the user indices and paperID
#rating = user_hash_map.select('user_idx', 'paperID')
rating = userLib.select('userID', 'paperID', 'libSize')

# # Split the string if all paperIDs into individual paperIDs and cast the datatype to int
# rating = rating.withColumn("papers", split(col("paperID"), ",\s*").cast(ArrayType(IntegerType())).alias("papers"))
# # Drop the column with the string of paperIDs
# rating = rating.drop("paperID")

# Add a column of 1's called ratings for the rated papers w.r.t to the user index
rating = rating.withColumn("rating", lit(1))

# Transform the distinct paperIDs dataframe to a list
paper_list = list(d_paper.select('paperID').toPandas()['paperID'])
# Map each distinct paper into int
paper_list = list(map(int, paper_list))

# Function to call in udf
def unrated(papers):
    # Transform the list of distinct papers and the list of rated papers of each user to a set
    # Substract the two sets to get the list of unrated papers for each user
    # Transform back to list
    unrated = list(set(paper_list) - set(papers))

    # Select n random papers from the unrated papers where n is the 
    # length of the rated papers list of each user
    unrated_papers = random.sample(unrated, len(papers))
            
    return unrated_papers


# udf to get a list of unrated papers with the length of rated papers for each user
get_unrated = udf(lambda x: unrated(x), ArrayType(IntegerType()))

# Add a new column of unrated papers for each user
rating = rating.withColumn("unrated", get_unrated(rating.paperID))

#Create a dataframe with the rated papers and their ratings of 1's
rated_df = rating.select("userID", f.explode("paperID").alias("paperID"), "rating", "libSize")

# Create a dataframe with the unrated papers
unrated_df = rating.select("userID", f.explode("unrated").alias("unrated"), "libSize")
# Add a column of ratings of 0's to the unrated papers
unrated_df = unrated_df.withColumn("rating", lit(0))
unrated_df = unrated_df.select("userID", col("unrated").alias("paperID"), "rating", "libSize")

# Union the rated and unrated dataframes and order by the user indices
rating_matrix = rated_df.union(unrated_df).orderBy("userID")

feature_df = rating_matrix.join(w_df, ["paperID"]).orderBy("userID")

window = Window.partitionBy(col("userID"))

feature_df_rated = feature_df.filter(feature_df.rating == 1).groupBy("userID", "paperID", "rating", "pages", "year", "type", "libSize").agg(f.avg("pages").over(window).alias("avgPages")).orderBy("userID")

feature_df_unrated = feature_df.filter(feature_df.rating == 0)
feature_df_unrated = feature_df_unrated.withColumn('avgPages', lit(" ").cast(StringType()))
feature_df_unrated = feature_df_unrated.select("userID", "paperID", "rating", "pages", "year", "type", "libSize", "avgPages")

y = feature_df_rated.select('userID', 'avgPages').distinct()

feature_df_unrated =  feature_df_unrated.drop("avgPages")

x = feature_df_unrated.join(y, 'userID')


model_df = feature_df_rated.union(x).orderBy("userID", "rating")
model_df1 = model_df.withColumn("pages", coalesce(model_df.pages, model_df.avgPages))

model_df1.show()

+--------------------+--------+------+-----+----+-----------+-------+--------+
|              userID| paperID|rating|pages|year|       type|libSize|avgPages|
+--------------------+--------+------+-----+----+-----------+-------+--------+
|589b870a611c25fa9...|11733005|     0|    7|2012|    article|      5|    17.0|
|589b870a611c25fa9...| 9045137|     0|   24|2007|    article|      5|    17.0|
|589b870a611c25fa9...|  115945|     0|   11|2005|    article|      5|    17.0|
|589b870a611c25fa9...| 8310458|     0|   15|2008|    article|      5|    17.0|
|589b870a611c25fa9...| 3728173|     0|    5|2007|    article|      5|    17.0|
|589b870a611c25fa9...| 1001231|     1| 17.0|2005|proceedings|      5|    17.0|
|589b870a611c25fa9...|12832332|     1|   29|2013|     inbook|      5|    17.0|
|589b870a611c25fa9...|  352713|     1|   12|2002|       misc|      5|    17.0|
|589b870a611c25fa9...| 1305474|     1| 17.0|2002|       misc|      5|    17.0|
|589b870a611c25fa9...|  956315|     1|   10|2006|   

In [54]:
stringIndexer = StringIndexer(inputCol="type", outputCol="typeIndex")
model = stringIndexer.fit(model_df1)
indexed = model.transform(model_df1)

encoder = OneHotEncoder(inputCol="typeIndex", outputCol="encodedType")
encoded = encoder.transform(indexed)
encoded.show()

+--------------------+--------+------+-----+----+-----------+-------+--------+---------+-------------+
|              userID| paperID|rating|pages|year|       type|libSize|avgPages|typeIndex|  encodedType|
+--------------------+--------+------+-----+----+-----------+-------+--------+---------+-------------+
|589b870a611c25fa9...| 8310458|     0|   15|2008|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...| 9045137|     0|   24|2007|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...| 3728173|     0|    5|2007|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...|  115945|     0|   11|2005|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...|11733005|     0|    7|2012|    article|      5|    17.0|      0.0|(3,[0],[1.0])|
|589b870a611c25fa9...| 1001231|     1| 17.0|2005|proceedings|      5|    17.0|      2.0|(3,[2],[1.0])|
|589b870a611c25fa9...|  352713|     1|   12|2002|       misc|      5|    

In [55]:
model_df1 = encoded.withColumn("pages", encoded["pages"].cast(DoubleType()))
model_df1 = model_df1.withColumn("year", model_df1["year"].cast(IntegerType()))

In [70]:
assembler = VectorAssembler(inputCols=["pages", "year", "libSize", "avgPages", "encodedType"], outputCol="features")

preprocessed_df = assembler.transform(model_df1)
preprocessed_df = preprocessed_df.select("userID", "paperID", col("rating").alias("label"),"features")

preprocessed_df.show()

+--------------------+--------+-----+--------------------+
|              userID| paperID|label|            features|
+--------------------+--------+-----+--------------------+
|589b870a611c25fa9...| 8310458|    0|[15.0,2008.0,5.0,...|
|589b870a611c25fa9...|11733005|    0|[7.0,2012.0,5.0,1...|
|589b870a611c25fa9...| 3728173|    0|[5.0,2007.0,5.0,1...|
|589b870a611c25fa9...| 9045137|    0|[24.0,2007.0,5.0,...|
|589b870a611c25fa9...|  115945|    0|[11.0,2005.0,5.0,...|
|589b870a611c25fa9...|12832332|    1|[29.0,2013.0,5.0,...|
|589b870a611c25fa9...| 1305474|    1|[17.0,2002.0,5.0,...|
|589b870a611c25fa9...| 1001231|    1|[17.0,2005.0,5.0,...|
|589b870a611c25fa9...|  352713|    1|[12.0,2002.0,5.0,...|
|589b870a611c25fa9...|  956315|    1|[10.0,2006.0,5.0,...|
|90f1a3e6fcdbf9bc5...|  956315|    0|[10.0,2006.0,5.0,...|
|90f1a3e6fcdbf9bc5...| 1305474|    0|[12.4,2002.0,5.0,...|
|90f1a3e6fcdbf9bc5...| 1001231|    0|[12.4,2005.0,5.0,...|
|90f1a3e6fcdbf9bc5...|12832332|    0|[29.0,2013.0,5.0,..

### Exercise 6. 3 (Classificatoin Algorithms) & Exercise 6. 4 (Model Evaluation)

In [80]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [140]:
def svm(df, features):
    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

    #Automatically identify categorical features, and index them.
    #We specify maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol=features, outputCol="indexedFeatures", maxCategories=4).fit(df)

    # Split the data into training and test sets
    (trainingData, testData) = df.randomSplit([0.8, 0.2])

    # Train a DecisionTree model.
    dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

    # Chain indexers and tree in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

    # Train model.  This also runs the indexers.
    model = pipeline.fit(trainingData)

    # Make predictions.
    predictions = model.transform(testData)

    # Select example rows to display.
    predictions.select("prediction", "label", features).show()

    # Select (prediction, true label) and compute test error
    evaluator =  RegressionEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="rmse")

    rmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))
    
svm(preprocessed_df, "features")

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       1.0|    1|[17.0,2002.0,5.0,...|
|       1.0|    0|[12.0,2002.0,5.0,...|
|       0.0|    1|[5.0,2007.0,5.0,1...|
+----------+-----+--------------------+

Root-mean-square error = 0.8164965809277261


In [135]:
def randomForest(df, features):
    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df)

    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer = VectorIndexer(inputCol=features, outputCol="indexedFeatures", maxCategories=4).fit(df)

    # Split the data into training and test sets
    (trainingData, testData) = df.randomSplit([0.8, 0.2])

    # Train a RandomForest model.
    rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                                   labels=labelIndexer.labels)

    # Chain indexers and forest in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

    # Train model.  This also runs the indexers.
    model = pipeline.fit(trainingData)

    # Make predictions.
    predictions = model.transform(testData)

    # Select example rows to display.
    predictions.select("predictedLabel", "label", features).show()

    evaluator =  RegressionEvaluator(
        labelCol="indexedLabel", predictionCol="prediction", metricName="rmse")

    rmse = evaluator.evaluate(predictions)
    print("Root-mean-square error = " + str(rmse))
    
randomForest(preprocessed_df, "features")

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|             0|    0|[5.0,2007.0,5.0,1...|
|             1|    0|[15.0,2008.0,5.0,...|
|             1|    0|[24.0,2007.0,5.0,...|
|             1|    0|[12.4,2005.0,5.0,...|
|             1|    1|[5.0,2007.0,5.0,1...|
+--------------+-----+--------------------+

Root-mean-square error = 0.7745966692414833


### Exercise 6. 5 (Additional Features)

In [120]:
################################### Vector representation for the papers ###########################################

# Extracting words from the papers and keeping "-" and "_"
tokenizer = RegexTokenizer(inputCol="words", outputCol="tokens", pattern="[a-zA-Z-_]+", gaps=False) 
# Built-in tokenizer
tokenized = tokenizer.transform(w_df)
tokenized = tokenized.select("paperID", "tokens")

# udf to remove "-" and "_" from the tokens
remove_hyphen_udf = udf(lambda x: [re.sub('[-|_]', '', word) for word in x], ArrayType(StringType()))
# Apply udf to the tokens
df = tokenized.withColumn('tokens', remove_hyphen_udf(col('tokens')))

# udf to remove words less than 3 letters
remove_short_words = udf(lambda x: [item for item in x if len(item) >= 3], ArrayType(StringType()))
# Apply udf to the tokens
df = df.withColumn('tokens', remove_short_words(col('tokens')))

# Built-in function to remove stopwords from our custom list
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered" , stopWords=stopwordList)
df = remover.transform(df)
df = df.select("paperID", "filtered")

# Apply stemming with NLTK
# Built-in class from NLTK
ps = PorterStemmer()
# udf to apply stemming
stemming = udf(lambda x: [ps.stem(item) for item in x], ArrayType(StringType()))
# apply udf to tokens
df = df.withColumn('stemmed', stemming(col('filtered')))
df = df.select("paperID", "stemmed")

# Create a new df to store the paperID and stemmed tokens
paper_terms = df

# Explode/Split the tokens in the list for each paperID and get the distinct tokens
df = df.select("paperID", f.explode("stemmed").alias("tokens")).distinct().orderBy("paperID")

# Assign count of 1 to each token w.r.t. the paperID since the tokens are distinct
df = df.groupBy("paperID","tokens").count()

# Get the number of distinct papers
num_papers = w_df.select("paperID").distinct().count()

# Get the value of ten percent of the number of papers
ten_percent = math.ceil(num_papers*.1)

# Create a new df with the tokens and count (without paperID)
df2 = df.select("tokens", "count")
# Count the number of papers containing the tokens
df2 = df2.groupBy("tokens").agg(f.collect_list("tokens").alias("duplicated_values"), f.sum("count").alias("count"))
# Filter out tokens that appeared in more than 10 percent of the papers
df2 = df2.drop("duplicated_values").orderBy((col("count")).desc()).filter(col("count") < ten_percent)
# Filter out tokens that appeared in less than 20 papers
# Limit the df to 1000 tokens
df2 = df2.filter(col("count") >= 20).limit(1000)
# Create a new df with terms and count
important_words = df2.select(col("tokens").alias("terms"), col("count"))
# Output the set of important words, T
#important_words.show()

In [121]:
# Create a new df where each term is replaced by a unique index that takes a value from the range between 0 and |T| − 1
df = important_words.withColumn("row_num", row_number().over(Window.orderBy("count"))-1)
# Create a df to store the indices and the corresponding terms
terms_index_hash = df.select(col("row_num").alias("index"), "terms")
#terms_index_hash.show()

num_terms = terms_index_hash.select("terms").distinct().count()
#print(num_terms)

In [122]:
p_terms = paper_terms.select("paperID", f.explode("stemmed").alias("terms"))

# Join p_terms with the terms_index_hash to replace the terms with the indices
joined_df = terms_index_hash.join(p_terms, ["terms"])
joined_df = joined_df.drop("index")

# Create a new df to compute the term frequency vectors
tf_df = joined_df
tf_df = tf_df.groupby("paperID").agg(f.concat_ws(", ", f.collect_list(tf_df.terms)).alias("terms"))
tf_df = tf_df.withColumn("terms_", split(col("terms"), ",\s*").cast(ArrayType(StringType())).alias("terms"))
tf_df = tf_df.drop("terms")
# tf_df is now a df with a column of paperID and a column of lists of the terms (unexploded)
#tf_df.show()

cv = CountVectorizer(inputCol="terms_", outputCol="vectors")
model = cv.fit(tf_df)
vector_df = model.transform(tf_df)
vector_df = vector_df.select("paperID", col("vectors").alias("term_frequency_sparse"))

# Term frequency vector df
vector_df.show()

+-------+---------------------+
|paperID|term_frequency_sparse|
+-------+---------------------+
| 159967| (82,[0,7,18,21,25...|
|2212959| (82,[1,4,21,40,64...|
| 333353| (82,[6,8,10,35,36...|
| 438129| (82,[30,59],[1.0,...|
| 166220| (82,[11,16,25,28,...|
|2883810| (82,[8,51,80],[1....|
|1288940| (82,[2,4,13,19,44...|
|5251453| (82,[0,6,13,14,19...|
|7515828| (82,[5,8,51,80],[...|
|2739852| (82,[0,2,3,17,21,...|
|5961524| (82,[10,20,46,51,...|
|    272| (82,[9,12,13,24,3...|
|6573750| (82,[9,23],[1.0,1...|
|  77265| (82,[6,10,20,67],...|
| 820297| (82,[15,43,57,68,...|
|1042553| (82,[7,24,39,41,4...|
|2883820| (82,[22,35,74],[1...|
|5434882| (82,[10,17,28,35,...|
|1332540|      (82,[68],[1.0])|
|1777140| (82,[0,1,7,18,32,...|
+-------+---------------------+
only showing top 20 rows



In [123]:
########################################## TF-IDF with built-in function ##########################################

from pyspark.ml.feature import HashingTF, IDF, Tokenizer

idf = IDF(inputCol="term_frequency_sparse", outputCol="features")
idfModel = idf.fit(vector_df)
rescaledData = idfModel.transform(vector_df)
tf_idf_built_in = rescaledData.select("paperID", "features")

#tf_idf_built_in.show()

# create the user profile using the tf_idf dataframe and the users' library dataframe
user_profile = tf_idf_built_in.join(user_df, ["paperID"]).orderBy("userID").select('userId', 'features')

# convert the dataframe to RDD to sum up the tf_idf vector of each user and then convert back to dataframe
user_profile = user_profile.rdd.mapValues(lambda v: v.toArray()).reduceByKey(lambda x, y: x + y).mapValues(lambda x: DenseVector(x))\
.toDF(["userId", "features"])

user_profile = user_profile.select("userId", col("features").alias("user_profile"))


In [128]:

# Convert dense vectors to sparse
def to_sparse(x):        
    # store the indices of non-zero elements
    nonzero_indices = np.nonzero(x)[0].tolist()
    # store the value of non-zero elements
    nonzero_counts = [num for num in x if num]
    # combine them to make a sparse vector
    sparse = SparseVector(num_terms, nonzero_indices, nonzero_counts)
    return sparse

to_sparse_udf = udf(lambda x: to_sparse(x), VectorUDT())

# Conver the feature vectors to sparse vectors
user_profile = user_profile.withColumn("user_profile", to_sparse_udf(col("user_profile")))

#user_profile.show()

# Join with the user profile with their unrated papers from unrated_df
df = unrated_df.join(tf_idf_built_in, ["paperID"]).join(user_profile, ["userID"])
df = df.select("userID", "paperID", "user_profile", col("features").alias("paper_profile"))
# df is now a dataframe with userID, paperID, user_profile, and paper_profile

# df.show()


def cos_sim(u, p):
    result = (np.dot(u, p))/(np.linalg.norm(u) * np.linalg.norm(p))
    result = result.item()
    return result

compute_sim = udf(cos_sim, FloatType())

sim_df = df.withColumn('Similarity', compute_sim(df.user_profile, df.paper_profile))
sim_df = sim_df.drop("user_profile", "paper_profile")

preprocessed_df_joined = preprocessed_df.join(sim_df, ["userID", "paperID"])

preprocessed_df_joined.show()

+--------------------+--------+-----+--------------------+-----------+
|              userID| paperID|label|            features| Similarity|
+--------------------+--------+-----+--------------------+-----------+
|90f1a3e6fcdbf9bc5...|  956315|    0|[10.0,2006.0,5.0,...| 0.04896834|
|589b870a611c25fa9...| 3728173|    0|[5.0,2007.0,5.0,1...|  0.0930714|
|90f1a3e6fcdbf9bc5...|  352713|    0|[12.0,2002.0,5.0,...|0.045270264|
|90f1a3e6fcdbf9bc5...| 1305474|    0|[12.4,2002.0,5.0,...|0.021175466|
|589b870a611c25fa9...|11733005|    0|[7.0,2012.0,5.0,1...| 0.08924326|
|589b870a611c25fa9...| 8310458|    0|[15.0,2008.0,5.0,...| 0.17026126|
|589b870a611c25fa9...|  115945|    0|[11.0,2005.0,5.0,...| 0.18160164|
|589b870a611c25fa9...| 9045137|    0|[24.0,2007.0,5.0,...|0.118237026|
|90f1a3e6fcdbf9bc5...| 1001231|    0|[12.4,2005.0,5.0,...|  0.0958606|
|90f1a3e6fcdbf9bc5...|12832332|    0|[29.0,2013.0,5.0,...| 0.18497433|
+--------------------+--------+-----+--------------------+-----------+



In [129]:
assembler = VectorAssembler(inputCols=["features", "Similarity"], outputCol="featuresAdded")

preprocessed_df_joined = assembler.transform(preprocessed_df_joined)
preprocessed_df_joined = preprocessed_df_joined.select("userID", "paperID", "label","featuresAdded")

preprocessed_df_joined.show()

+--------------------+--------+-----+--------------------+
|              userID| paperID|label|       featuresAdded|
+--------------------+--------+-----+--------------------+
|90f1a3e6fcdbf9bc5...|  956315|    0|[10.0,2006.0,5.0,...|
|589b870a611c25fa9...| 3728173|    0|[5.0,2007.0,5.0,1...|
|90f1a3e6fcdbf9bc5...|  352713|    0|[12.0,2002.0,5.0,...|
|90f1a3e6fcdbf9bc5...| 1305474|    0|[12.4,2002.0,5.0,...|
|589b870a611c25fa9...|11733005|    0|[7.0,2012.0,5.0,1...|
|589b870a611c25fa9...| 8310458|    0|[15.0,2008.0,5.0,...|
|589b870a611c25fa9...|  115945|    0|[11.0,2005.0,5.0,...|
|589b870a611c25fa9...| 9045137|    0|[24.0,2007.0,5.0,...|
|90f1a3e6fcdbf9bc5...| 1001231|    0|[12.4,2005.0,5.0,...|
|90f1a3e6fcdbf9bc5...|12832332|    0|[29.0,2013.0,5.0,...|
+--------------------+--------+-----+--------------------+



In [ ]:
svm(preprocessed_df_joined, "featuresAdded")

In [ ]:
randomForest(preprocessed_df_joined, "featuresAdded")